In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print(tf.__version__)

2.2.0


In [3]:
class Linear(keras.layers.Layer):
    def __init__(self,units=32,input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            w_init(shape=(input_dim,units),dtype='float32'),
            trainable = True
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            b_init(shape=(units,),dtype='float32'),
            trainable=True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs,self.w)+self.b

In [4]:
x = tf.ones((2,2))

In [5]:
my_linear = Linear(4, 2)

In [6]:
y = my_linear(x)

In [7]:
print(y)

tf.Tensor(
[[-0.01594551  0.06607145  0.03315714  0.06755836]
 [-0.01594551  0.06607145  0.03315714  0.06755836]], shape=(2, 4), dtype=float32)


In [8]:
my_linear.weights

[<tf.Variable 'Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.0280911 ,  0.0352839 ,  0.04287689,  0.04171585],
        [ 0.01214559,  0.03078755, -0.00971975,  0.02584251]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

# 还可以使用一种更快捷的方式为层添加权重：add_weight()方法

In [ ]:
class Linear(keras.layers.Layer):
    def __init__(self,units=32,input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape = (input_dim, units),
            initializer='random_normal',
            trainable = True
        )
        self.b = self.add_weight(
            shape = (units,),
            initializer='zeros',
            trainable = True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs,self.w)+self.b

   <font size=5> 注意trainable参数 </font>
   
   
   <font size=5> 将权重创建推迟到得知输入的形状之后 </font>

    
   <font size=5> 在层的build(self, input_shape)方法中创建层的权重 </font>

In [8]:
class Linear(keras.layers.Layer):
    def __init__(self,units=32):
        super(Linear, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
            shape = (input_shape[-1], self.units),
            initializer='random_normal',
            trainable = True
        )
        self.b = self.add_weight(
            shape = (self.units,),
            initializer='zeros',
            trainable = True
        )
        
    def call(self, inputs):
        return tf.matmul(inputs,self.w)+self.b

In [9]:
my_linear = Linear(4)

In [10]:
my_linear.weights

[]

In [11]:
x= tf.ones((2,2))

In [12]:
my_linear(x)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.01932704, -0.14236471, -0.03288966,  0.00870322],
       [-0.01932704, -0.14236471, -0.03288966,  0.00870322]],
      dtype=float32)>

In [13]:
my_linear.weights

[<tf.Variable 'linear_1/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.0193696 , -0.08279815, -0.00830636,  0.05509334],
        [-0.03869664, -0.05956656, -0.0245833 , -0.04639013]],
       dtype=float32)>,
 <tf.Variable 'linear_1/Variable:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

# 层可递归组合
   <font size=5> 如果将一个层实例分配为另一个层的特性，则外部层将开始跟踪内部层的权重 </font>
   
   <font size=5>我们建议在__init__()方法中创建此类子层（由于子层通常具有构建方法，它们将与外部层同时构建） </font>


In [14]:
class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.lin_1 = Linear(32)
        self.lin_2 = Linear(64)
        self.lin_3 = Linear(1)
        
    def call(self, inputs):
        x = self.lin_1(inputs)
        x = tf.nn.relu(x)
        x = self.lin_2(x)
        x = tf.nn.relu(x)
        x = self.lin_3(x)
        return x

In [15]:
class MLPBlock_1(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.lin_1 = tf.keras.layers.Dense(32)
        self.lin_2 = tf.keras.layers.Dense(64)
        self.lin_3 = tf.keras.layers.Dense(32)
        
    def call(self, inputs):
        x1 = self.lin_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.lin_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.lin_3(x2)
        return tf.concat([x1,x3])

<font size=5>通常使用Layer类来定义内部计算块，使用Modlel类来定义外部模型，即训练的对象</font>


<font size=5>Model类具有与Layer相同的API，但有如下区别：</font>

+ <font size=4>它会公开内置训练、评估和预测循环（model.fit(), model.evaluate(), model.predict()）</font>
+ <font size=4>它会通过model.layers属性公开其内部层的列表</font>
+ <font size=4>它会公开保存和序列化API(save(),save_weights()...)</font>

In [17]:
class MLPBlock_model(keras.Model):
    def __init__(self):
        super(MLPBlock_model, self).__init__()
        self.lin_1 = tf.keras.layers.Dense(32)
        self.lin_2 = tf.keras.layers.Dense(64)
        self.lin_3 = tf.keras.layers.Dense(32)
        
    def call(self, inputs):
        x1 = self.lin_1(inputs)
        x1 = tf.nn.relu(x1)
        x2 = self.lin_2(x1)
        x2 = tf.nn.relu(x2)
        x3 = self.lin_3(x2)
        return tf.concat([x1,x3])

In [19]:
model = MLPBlock_model()

In [20]:
model.save

<bound method Network.save of <__main__.MLPBlock_model object at 0x7ff169af5978>>

<font size=5>因此，如果您想知道“我应该用Layer类还是Model类？”</font>

<font size=5>请问自己：我是否需要在它上面调用fit()?</font>

<font size=5>我是否需要在它上面调用save()?</font>

<font size=5>如果是，则使用Model。如果不是（要么因为您的类只是更大系统中的一个块，要么因为您正在自己编写训练和保存代码），则使用Layer.</font>